<div style="padding:8px 0px 8px 15px;border-left:3px solid #B51F1F;background-color:#F3F3F3;">
    <span style="font-weight:bold;text-decoration:underline">
        Mastering Game theory
    </span>
    <br/>
</div>

In [76]:
import json
participants_info = json.load(open('participants.json'))
id2group = {x['id']:x['group'] for x in participants_info}
id2group['psi'] = 0
id2group['ips'] = 1
import time

In [84]:
import ipywidgets as widgets
from IPython.display import display, IFrame, update_display
import Casino as c

# Parameters
player_money = 10
host_reward = 3
bet_lower_limit = 2
bet_upper_limit = 4
timestamps = []
# Environment
casino = c.Casino(host_reward, bet_lower_limit, bet_upper_limit)
player = c.Player(player_money, casino=casino)
agents = [c.BraggerAgent(casino), c.CowardAgent(casino), c.RandomAgent(casino), c.SmartAgent(casino)]
playthrough = c.Playthrough(casino, player, agents, True)

def reset_env():
    global casino, player, agents, playthrough
    casino = c.Casino(host_reward, bet_lower_limit, bet_upper_limit)
    player = c.Player(player_money, casino=casino)
    agents = [c.BraggerAgent(casino), c.CowardAgent(casino), c.RandomAgent(casino), c.SmartAgent(casino)]
    playthrough = c.Playthrough(casino, player, agents, True)
    
def make_html_line(lines:list):
    html = "<p>"
    for line in lines:
        html += line + "<br>"
    html += "</p>"
    return html

def make_html_paragraph(line:str):
    paragraph_html = """
    <!DOCTYPE html>
    <html>
    <head>
        <style>
            body {
                font-family: Arial, sans-serif;
                font-size: 14px;
            }
        </style>
    </head>
    <body>"""
    para = make_html_line(line.split("\n"))
    paragraph_html_end = """
    </body>
    </html>
    """
    return paragraph_html + para + paragraph_html_end

import time
import threading
reset_env()

page_a_content = widgets.HTML(make_html_paragraph(playthrough.get_intro_str()))

button_continue = widgets.Button(description='Continue')
button_start = widgets.Button(description='Next')
output = widgets.Output()
ui = widgets.VBox()

button_end = widgets.Button(description='proceed')

round_number = 0
bet = None
choice = None

main_handle = None

timer_widget = widgets.FloatProgress(
    value=0,
    min=0,
    max=30,
    description='Time Limit:',
    bar_style='info',
    orientation='horizontal',
    style={'bar_color': "green"}
)
time_begin = 0
timer_start = False

# get color from green to red
def get_color(value:float, min:float, max:float):
    return f"#" + f"{int(255 * (value - min) / (max - min)) :02x}" + f"{int(255 * (max - value) / (max - min)) :02x}" + f"00"

def on_button_continue_clicked(b):
    global round_number
    global main_handle
    global timer_start
    timestamps.append({'ps_submit':time.time()})
  #  output.clear_output()  # Clear the previous output
    timer_start = False
    bet_value = bet.value
    if choice.value == "High":
        choice_value = 1
    elif choice.value == "Low":
        choice_value = 0
    playthrough.play_round(round_number, bet_value, choice_value)
    page_round_result_content = widgets.HTML(make_html_paragraph(playthrough.get_round_result_str(round_number)))
    round_number += 1
    if round_number >= playthrough.get_rounds():
        page_conclusion_content = widgets.HTML(make_html_paragraph(playthrough.get_conclusion_str()))
        ui.children = [page_round_result_content, page_conclusion_content,button_end]
    else:
        ui.children = [page_round_result_content,button_start]
    

def on_button_clicked(b):
    global round_number 
    global bet
    global choice
    global timer_start
    global time_begin
    global main_handle
    with output:
        timestamps.append({'ps_start':time.time()})
        #output.clear_output()  # Clear the previous output
        page_round_content = widgets.HTML(make_html_paragraph(playthrough.get_round_str(round_number)))
        bets = player.get_availiable_bets()
        bet = widgets.BoundedIntText(
            value=bets[0],
            min=min(bets),
            max=max(bets),
            step=1,
            description='Bet:',
            disabled=False
        )
        choice = widgets.RadioButtons(
            options=['High', 'Low'],
            description='Choice:',
            disabled=False
        )
        time_begin = time.time()
        timer_widget.value = 0
        time_limit = playthrough.get_round_time_limit(round_number)
        timer_widget.max = time_limit
        timer_start = True
        ui.children = [page_round_content, bet, choice, timer_widget, button_continue]
        def update_time():
            global time_begin
            global button_continue
            button_continue.disabled = True
            button_continue.description = "Thinking..."
            while timer_start == False:
                time.sleep(0.1)
            while timer_widget.value < timer_widget.max:
                #timer_widget.style={'bar_color': get_color(timer_widget.value, timer_widget.min, timer_widget.max)}
                if timer_widget.value >= 5:
                    button_continue.disabled = False
                    button_continue.description = "Bet!"
                timer_widget.value = time.time() - time_begin
                time.sleep(0.1)
                
                if timer_start == False:
                    return

            button_continue.click()

        thread = threading.Thread(target=update_time)
        thread.start()
        

button_continue.on_click(on_button_continue_clicked)
button_start.on_click(on_button_clicked)

# Initially display Page A content and the button
ui.children = [page_a_content, button_start]

import ipywidgets as widgets
from IPython.display import display, IFrame, update_display, YouTubeVideo

Intro_content = widgets.HTML("""
Welcome to <b>Mastering Game Theory: Nash Equilibrium and the Prisoner's Dilemma</b>
</br></br><b>Project Team:</b></br> We, Aziz Dhouib, Hyeongdon Moon, Menjie Zou and Sofiya Malamud, currently master’s students at EPFL, are conducting a research project for our course "Digital Education and Learning Analytics," under the guidance of Professor Pierre Dillenbourg.
</br></br><b>Study Purpose:</b></br> Our study aims to evaluate two pedagogical approaches, I-PS (Instruction followed by Problem Solving) and PS-I (Problem Solving followed by Instruction), specifically in the context of musical education.
</br></br><b>Anonymity Assurance:</b></br> Participant anonymity is paramount in our study. We ensure that all data collection and storage will be conducted without personal identification.
</br></br><b>Participant Tasks:</b></br> Participants will engage in several activities, including taking tests to gauge learning outcomes, reading instructional materials, and applying their knowledge to harmonize melodies on a specially designed website for this study.
</br></br><b>Risk and Comfort:</b></br> The study involves standard educational activities, posing a minimal risk. Should any discomfort arise, participants are encouraged to contact our course instructor, Pierre Dillenbourg, at pierre.dillenbourg@epfl.ch.
</br></br><b>Voluntary Participation:</b></br> Participation in this study is entirely voluntary and non-compulsory. Participants have the freedom to withdraw at any stage or to skip any questions they prefer not to answer. Participation or non-participation will not influence course grades or assessments for EPFL students.
</br></br><b>Withdrawal from Study:</b></br> Participants can opt out of the study at any time. Due to the anonymous nature of the data, we cannot remove specific data post-collection.
0</br></br><b>Study Benefits</b></br>: Participants will gain practical skills in basic melody harmonization using the circle of fifths, a foundational concept in music theory.
</br></br><b>Data Utilization:</b></br> The data will contribute to a project report and presentation in the "Digital Education and Learning Analytics" course.
</br></br><b>Data Storage and Security:</b></br> In compliance with Swiss Federal Law on data protection (RS 235.1), all collected data will be securely stored. Access to this data will be limited to the project team and the Research Ethics Committee, and all data will be deleted after the course's conclusion.
</br></br><b>Health and Safety:</b></br> As the study involves standard educational activities, the risk of harm is minimal. Any health issues directly resulting from the study and attributable to EPFL will be covered by EPFL's general liability insurance (policy number 30/5.006.824 of Baloise Insurance). Participants' personal health or accident insurance will also apply.
</br></br>
<b> Accept and proceed with this experiment by entering your provided id and pressing the <i>Proceed</i> button below. </b>
""")
Intro_id = widgets.Text(description="Your ID: ",disabled=False)

Test_announce = widgets.HTML('<b>Now the test will begin.</b></br>Please <b>log in to moodle with your id ans password.</b> </br> After log in, press <i>proceed</i> button.')
PS_announce = widgets.HTML('Now you will be asked to do an excercise related to our lesson.')
Lecture_announce = widgets.HTML('Now the lecture about our topic will begin. Please keep your attention to the lecture.')
Post_test_announce = widgets.HTML('<b>Final test</b></br> Now you will be asked to solve problems similar to the ones you saw at the beginning. </br>')
Finish_announce = widgets.HTML('Thank you for your participant. </br> All session is done.')

button_1 = widgets.Button(description='Understand. Proceed!') # Welcome message. Ask to register 
button_2 = widgets.Button(description='Proceed to test') # Notice the quick test.
button_3 = widgets.Button(description='Next question') # 1->2
button_4 = widgets.Button(description='Next question') # 2-> 3
button_5 = widgets.Button(description='Next question') # 3-> 4
button_6 = widgets.Button(description='Finish test') # 4-> PS if psi, else 4->i announce
# button_end PS -> I announce if psi, else PS->Post test notice
button_ps_start = widgets.Button(description='Start challenge!')
button_8 = widgets.Button(description='Proceed') # I announce -> I
button_9 = widgets.Button(description='Proceed') # I -> Post test notice if psi, else I->PS_announce
button_10 = widgets.Button(description='Proceed') # I -> Post test begin
button_post_1 = widgets.Button(description='Proceed') # Post-test Q2 begin.
button_post_2 = widgets.Button(description='Proceed') # Post-test Q3 begin.
button_post_3 = widgets.Button(description='Proceed') # Post-test Q4 begin.
button_post_4 = widgets.Button(description='Proceed') # Experiment end.

Instruction = YouTubeVideo('nf7bebfjiCQ')
Login = IFrame("https://www.cs411-moodle.com/login/index.php", 800, 550)
Q1 = IFrame("https://www.cs411-moodle.com/h5p/embed.php?url=https%3A%2F%2Fwww.cs411-moodle.com%2Fpluginfile.php%2F10505%2Fmod_h5pactivity%2Fpackage%2F0%2Fmultiple-choice-94.h5p&amp;component=mod_h5pactivity", 1530, 450)
Q2 = IFrame("https://www.cs411-moodle.com/h5p/embed.php?url=https%3A%2F%2Fwww.cs411-moodle.com%2Fpluginfile.php%2F10506%2Fmod_h5pactivity%2Fpackage%2F0%2Fmultiple-choice-95.h5p&amp;component=mod_h5pactivity", 1530, 450)
Q3 = IFrame("https://www.cs411-moodle.com/h5p/embed.php?url=https%3A%2F%2Fwww.cs411-moodle.com%2Fpluginfile.php%2F10507%2Fmod_h5pactivity%2Fpackage%2F0%2Fmultiple-choice-96.h5p&amp;component=mod_h5pactivity", 1530, 450)
Q4 = IFrame("https://www.cs411-moodle.com/h5p/embed.php?url=https%3A%2F%2Fwww.cs411-moodle.com%2Fpluginfile.php%2F10508%2Fmod_h5pactivity%2Fpackage%2F0%2Fmultiple-choice-97.h5p&amp;component=mod_h5pactivity", 1530, 450)

post_q1 = IFrame("https://www.cs411-moodle.com/h5p/embed.php?url=https%3A%2F%2Fwww.cs411-moodle.com%2Fpluginfile.php%2F10681%2Fmod_h5pactivity%2Fpackage%2F0%2Fmultiple-choice-94%25282%2529%25281%2529.h5p&amp;component=mod_h5pactivity", 1530, 450)
post_q2 = IFrame("https://www.cs411-moodle.com/h5p/embed.php?url=https%3A%2F%2Fwww.cs411-moodle.com%2Fpluginfile.php%2F10682%2Fmod_h5pactivity%2Fpackage%2F0%2Fmultiple-choice-103.h5p&amp;component=mod_h5pactivity", 1530, 450)
post_q3 = IFrame("https://www.cs411-moodle.com/h5p/embed.php?url=https%3A%2F%2Fwww.cs411-moodle.com%2Fpluginfile.php%2F10683%2Fmod_h5pactivity%2Fpackage%2F0%2Fmultiple-choice-104.h5p&amp;component=mod_h5pactivity", 1530, 450)
post_q4 = IFrame("https://www.cs411-moodle.com/h5p/embed.php?url=https%3A%2F%2Fwww.cs411-moodle.com%2Fpluginfile.php%2F10684%2Fmod_h5pactivity%2Fpackage%2F0%2Fsingle-choice-set-105.h5p&amp;component=mod_h5pactivity", 1530, 450)

def page1():
    with output:
        timestamps.append(time.time())
        display(Intro_content, Intro_id, button_1)
        
def page2(b):
    if Intro_id.value in id2group:
        timestamps.append(time.time())
        with output:
            output.clear_output()
            display(Test_announce, Login,button_2)
    else:
        invalid = widgets.Label('Enter valid id ')
        with output:
            display(invalid)
        
def page3(b):
    with output:
        timestamps.append(time.time())
        output.clear_output()
        display(Q1, button_3)
        
def page4(b):
    with output:
        timestamps.append(time.time())
        output.clear_output()
        display(Q2, button_4)      
        
def page5(b):
    with output:
        timestamps.append(time.time())
        output.clear_output()
        display(Q3, button_5)      
        
def page6(b):
    with output:
        timestamps.append(time.time())
        output.clear_output()
        display(Q4, button_6)    

def page_ps(b):    
    with output:
        timestamps.append(time.time())
        output.clear_output()
        display(ui)   
    
        
def page7(b):
    with output:
        timestamps.append(time.time())
        if id2group[Intro_id.value] == 0: # PS-I
            output.clear_output()
            display(PS_announce,button_ps_start)    
        else:
            output.clear_output()
            display(Lecture_announce, button_8)        
        
def page8(b):
    with output:
        if id2group[Intro_id.value] == 0: # PS-I
            timestamps.append(time.time())
            output.clear_output()
            display(Lecture_announce, button_8)           
        else:            
            timestamps.append(time.time())
            output.clear_output()
            display(Post_test_announce, button_10)

def page9(b):
    with output:
        timestamps.append(time.time())
        output.clear_output()
        display(Instruction, button_9)
        
def page_post(b):
    with output:
        timestamps.append(time.time())
        if id2group[Intro_id.value] == 0: # PS-I
            output.clear_output()
            display(Post_test_announce, button_10)
        else:
            output.clear_output()
            display(PS_announce,button_ps_start)   
            

def page_p1(b):
    with output:
        timestamps.append(time.time())
        output.clear_output()
        display(post_q1,button_post_1)

def page_p2(b):
    with output:
        timestamps.append(time.time())
        output.clear_output()
        display(post_q2,button_post_2)
        
def page_p3(b):
    with output:
        timestamps.append(time.time())
        output.clear_output()
        display(post_q3,button_post_3)
        
def page_p4(b):
    with output:
        timestamps.append(time.time())
        output.clear_output()
        display(post_q4,button_post_4)
        
def page_last(b):
    with output:
        timestamps.append(time.time())
        json.dump({Intro_id.value:timestamps},open(f'datas/{Intro_id.value}.json','w'))
        output.clear_output()
        display(Finish_announce)
    
button_1.on_click(page2)
button_2.on_click(page3)
button_3.on_click(page4)
button_4.on_click(page5)
button_5.on_click(page6)
button_6.on_click(page7)
button_end.on_click(page8)
button_ps_start.on_click(page_ps)
button_8.on_click(page9)
button_9.on_click(page_post)
button_10.on_click(page_p1)
button_post_1.on_click(page_p2)
button_post_2.on_click(page_p3)
button_post_3.on_click(page_p4)
button_post_4.on_click(page_last)

output.clear_output()
with output:
    display(Intro_content, Intro_id, button_1)
display(output)

Output()